# Omnibot Trajectory
* Run in the scene workshop1.ttt in CoppeliaSim

In [1]:
import sim
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Robot (given code)

In [4]:
class robot():
    
    def __init__(self, frame_name, motor_names=[], client_id=0):  
        # If there is an existing connection
        if client_id:
                self.client_id = client_id
        else:
            self.client_id = self.open_connection()
            
        self.motors = self._get_handlers(motor_names) 
        
        # Robot frame
        self.frame =  self._get_handler(frame_name)
            
        
    def open_connection(self):
        sim.simxFinish(-1)  # just in case, close all opened connections
        self.client_id = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)  # Connect to CoppeliaSim 
        
        if clientID != -1:
            print('Robot connected')
        else:
            print('Connection failed')
        return clientID
        
    def close_connection(self):    
        sim.simxGetPingTime(self.client_id)  # Before closing the connection to CoppeliaSim, make sure that the last command sent out had time to arrive.
        sim.simxFinish(self.client_id)  # Now close the connection to CoppeliaSim:
        print('Connection closed')
    
    def isConnected(self):
        c,result = sim.simxGetPingTime(self.client_id)
        # Return true if the robot is connected
        return result > 0         
        
    def _get_handler(self, name):
        err_code, handler = sim.simxGetObjectHandle(self.client_id, name, sim.simx_opmode_blocking)
        return handler
    
    def _get_handlers(self, names):
        handlers = []
        for name in names:
            handler = self._get_handler(name)
            handlers.append(handler)
        
        return handlers

    def send_motor_velocities(self, vels):
        for motor, vel in zip(self.motors, vels):
            err_code = sim.simxSetJointTargetVelocity(self.client_id, 
                                                      motor, vel, sim.simx_opmode_streaming)      
            
    def set_position(self, position, relative_object=-1):
        if relative_object != -1:
            relative_object = self._get_handler(relative_object)        
        sim.simxSetObjectPosition(clientID, self.frame, relative_object, position, sim.simx_opmode_oneshot)
        
    def simtime(self):
        return sim.simxGetLastCmdTime(self.client_id)
    
    def get_position(self, relative_object=-1):
        # Get position relative to an object, -1 for global frame
        if relative_object != -1:
            relative_object = self._get_handler(relative_object)
        res, position = sim.simxGetObjectPosition(self.client_id, self.frame, relative_object, sim.simx_opmode_blocking)        
        return array(position)
    
    def get_object_position(self, object_name):
        # Get Object position in the world frame
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_blocking)
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, -1, sim.simx_opmode_blocking)
        return array(position)
    
    def get_object_relative_position(self, object_name):        
        # Get Object position in the robot frame
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_blocking)
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, self.frame, sim.simx_opmode_blocking)
        return array(position)

# Making the Robot Follow the Disk	

In [6]:
# Joint name in coppelia
motor_names = ['Omnirob_FLwheel_motor', 'Omnirob_FRwheel_motor', 'Omnirob_RRwheel_motor', 'Omnirob_RLwheel_motor']
r = robot('Omnirob', motor_names)  # Create an instance of our robot

# Initial locations
robot_position = r.get_position()
print('Robot position: (%.2f, %.2f) '%(robot_position[0], robot_position[1]))

desired_position = r.get_object_position('Sphere1')
print('Desired position: (%.2f, %.2f) '%(desired_position[0], desired_position[1]))

relative_position = r.get_object_relative_position('Sphere1')
print('Relative position: (%.2f, %.2f) '%(relative_position[0], relative_position[1]))


# Virtual disk - add a 1 unit disk with minimal density and collision off
disk = robot('Disc', client_id=r.client_id)


# Time duration
# tf = 5
# Time interval
time_steps = 10
###
# These arrays contain the 3rd degree polynomial coefficients at each step,
# based on the spline trajectory calculated from the given "trajectories.ipynb"
# from class / GitHub repository.
#
# They are stored as tuples, where the first element is the coefficient for the trajectory in the X direction,
# and the second element is the coefficient for the trajectory in the Y direction.
###
a0 = np.array([(7.625, 8.55), 
               (0.4,8.05), 
               (4, 7.5),
               (1.925,2.225), 
               (-3.675, 3.4),
               (-3.675, 7.0),
               (0,0),
               (4.8,-2.5), 
               (9.5,-4.05),
               (6.125, -6.15),
               (0.5, -7.0),
               (-1.0, -3.0),
               (-3.2, -8.5),])
a1 = np.array([(-0.025, 0), 
               (0.5,0.01),  
               (-0.25, -0.25), 
               (-1.0,0), 
               (-0.001,2.0),
               (0.1,-0.1),
               (0.15,-0.65),
               (0.65,-0.1),
               (-0.25, -0.75),
               (-1.55, -0.5),
               (0,0.5),
               (-0.7,-0.7),
               (-0.01, 0)])
a2 = np.array([(-0.26175, -0.016), 
               (0.03300000000000001,0.006499999999999979), 
               (0.08775000000000001,-0.10825), 
               (0.032100000000000024, -0.16475), 
               (-0.0098,-0.282), 
               (0.07524999999999998, -0.125),
               (0.04899999999999999, 0.065),
               (0.03600000000000001, 0.048500000000000015),
               (0.10375, 0.13699999999999996),
               (0.14125, 0.02450000000000003),
               (0.025, 0.09),
               (0.07499999999999998,-0.025),
               (-0.10150000000000002,0.0)])
a3 = np.array([(0.0192, 0.0011000000000000014), 
               (-0.0047,-0.0012999999999999989), 
               (-0.00835,0.00805), 
               (0.0011900000000000014, 0.01765), 
               (0.0009900000000000002,0.011800000000000001), 
               (-0.004849999999999999, 0.0065),
               (-0.0015999999999999996, -0.0025),
               (-0.0054, -0.0054),
               (-0.01125, -0.0083),
               (-0.00425, 0.0016999999999999993),
               (-0.004, -0.01),
               (-0.0026999999999999993,0.004),
               (0.0068000000000000005,0.0)])

waypoints = 13

for i in range(waypoints):
    for t in range(16 * time_steps):
        t = t / 16
        x_traj = a0[i][0] + (a1[i][0] * t) + (a2[i][0] * (t**2)) + (a3[i][0] * (t**3)) # 0 for x
        y_traj = a0[i][1] + (a1[i][1] * t) + (a2[i][1] * (t**2)) + (a3[i][1] * (t**3)) # 1 for y
        x_vel = a1[i][0] + 2 * a2[i][0] * t + 3 * a3[i][0] * t**2
        y_vel = a1[i][1] + 2 * a2[i][1] * t + 3 * a3[i][1] * t**2

        disk.set_position([x_traj, y_traj, -1])


        # Sensing
        robot_position = r.get_position()


        # Trajectory tracker
        u = 15 * ((x_traj, y_traj, 0) - robot_position) + [x_vel, y_vel, 0]

        vx, vy, vz = u
        r.send_motor_velocities([-vy - vx, vy - vx, vy + vx, -vy + vx])

        time.sleep(0.02)


r.close_connection() 

NameError: name 'clientID' is not defined